In [19]:
import numpy as np
import torch
import cv2

from torch import nn

import  matplotlib.pyplot as plt
import torch.utils.data as Data
import torch.optim as optim

In [ ]:
#read dataset
trainX = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/train/train/trainX.pt')
trainY = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/train/train/trainY.pt')

In [ ]:
#split data set
rgb_images,depth_images,file_ids = trainX

In [ ]:
#show three different view image
def show_img_three_view(rgb_image):
    for i in range(3):
        one_image = rgb_image[i].numpy() 
        plt.imshow(one_image.transpose(2, 1, 0))
        plt.show()

In [ ]:
show_img_three_view(rgb_images[0])

In [ ]:
rgb_images[0].numpy().transpose(0,3, 2, 1).shape


In [ ]:
from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
#define train data set
class Trainset(Dataset):
    def __init__(self, rgb_images,labels):
        
        
        self.rgb_images = rgb_images
        self.labels = labels

    def __getitem__(self, index):
        return self.rgb_images[index].float(),self.labels[index].float()
        

    def __len__(self):
        return len(self.labels)
    
    def len(self):
        return len(self.rgb_images)


In [ ]:
#construct train set
dataset = Trainset(rgb_images,trainY[0])

In [ ]:
x,y = dataset.__getitem__(1)


In [ ]:
batch1 = rgb_images[:2]

In [ ]:
batch1.shape

In [ ]:
batch1[:,:,0,:,:].shape,batch1[:,:,1,:,:].shape,batch1[:,:,-1,:,:].shape

In [ ]:
#define train loader
train_loader = Data.DataLoader(dataset=dataset, batch_size=128, shuffle=True)
#or batch_size = 64

In [ ]:
class MyCRNN(nn.Module):
    #build my cnn and rnn model for predict data set
    def __init__(self):
        super(MyCRNN, self).__init__()
        self.cov1 = nn.Sequential( 
                    nn.Conv2d(in_channels=3, #input height
                            out_channels=16, #n_filter
                            kernel_size=5, #filter size
                            stride=1, #filter step
                            padding=2 
                            ), #output shape (16,28,28)
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2) #output shape (16,14,14)

                )
        self.cov2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2), #output shape (32,7,7)
                                        nn.ReLU(),
                                        nn.MaxPool2d(2)
        )

        self.cov3 = nn.Sequential(
            nn.Conv2d(32, 32, 2, 1, 2), #output shape (32,7,7)
                                        nn.ReLU(),
                                        nn.MaxPool2d(2,2),nn.Dropout(0.5)
        )

        self.myCnnNet = nn.Sequential(
            self.cov1,self.cov2,self.cov3,nn.Sequential(nn.Linear(29, 120), nn.Linear(120, 84)),
            #    nn.Linear(128, 12)
            
        )

        self.lstm = nn.LSTM(input_size=84, hidden_size=128, num_layers=2,
                                    batch_first=True)

        self.myPredictNet = nn.Sequential(
            
            
            nn.Linear(128, 64),nn.Linear(64, 12)
            
        )


    def forward(self, x):
        cnn_output_list = list()
        for t in range(batch1.size(1)):
            cnn_output_list.append( self.myCnnNet(x[:, t, :, :, :]))
        x = torch.stack(tuple(cnn_output_list), dim=1)
        out,h = self.lstm(x.view(x.size(0),-1,84))
        x = out[:, -1, :]
        return self.myPredictNet(x)

In [ ]:
myNet = MyCRNN()

In [ ]:
if torch.cuda.is_available():
    myNet.to('cuda:0')


In [ ]:
criterion = nn.MSELoss()
#optimizer = optim.SGD(myNet.parameters(), lr=0.001, momentum=0.9)
#Above is the SGD method function, other things holding constant
optimizer = torch.optim.Adam(myNet.parameters(), lr=0.001)  

In [ ]:
#train data
epoch = 30
for epoch in range(epoch):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.to('cuda:0')
            labels = labels.to('cuda:0')
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = myNet(inputs)
        
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'epoch [{epoch + 1}] train loss: {running_loss / i:.5f}')
        

In [ ]:
# Evaluate testX

In [ ]:
testX = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/test/test/testX.pt')

In [ ]:
train_rgb_images,depth_images,file_ids = testX

In [ ]:
prdict_arr = []

In [ ]:

myNet.eval()
for i in train_rgb_images:
    batch1 = torch.from_numpy(np.expand_dims(i.numpy(),axis=0)).float()
    if torch.cuda.is_available():
        batch1 = batch1.to('cuda:0')
    pos = myNet(batch1)
    prdict_arr.append(pos.to('cpu').detach().numpy())

In [ ]:
#output csv file
f = open('./submission.csv','a+',encoding='utf-8')
line = 'ID,FINGER_POS_1,FINGER_POS_2,FINGER_POS_3,FINGER_POS_4,FINGER_POS_5,FINGER_POS_6,FINGER_POS_7,FINGER_POS_8,FINGER_POS_9,FINGER_POS_10,FINGER_POS_11,FINGER_POS_12\n'
f.write(line)
for predict_Y,file_id in zip(prdict_arr,file_ids):

    line =  str(file_id) + ',' +  ','.join([str(round(i,4)) for i in predict_Y[0]]) 
    f.write(line + '\n')
    
f.close()